## Locations of Training Programs & Schools
This code visualizes the locations of the schools BY serves (each school that we've hosted a CTD fair) & its training programs. It will print the number of schools & trainings visualized at the bottom.

### What you need
You will need the pandas, folium, and geopy libraries, along with Python 3, on your computer. <br>
I use the "Trainings and Locations" & "Final 2021-2022 CTD Report" Salesforce reports. The ones I put in this file were pulled July 2022. <br>
For trainings, as long as there are "Training: Training Name", "Location (Latitude)", and "Location (Latitude)" columns, the code will work. <br>
For schools, as long as there are "Career Test Drive Name" and "School Address" columns, the code will work. Note that the code will delete entries that do not have full addresses (and therefore not show them on the map), so ensure that the report is complete. <br>
Export both reports as a CSV with ISO-8859 Encoding. If the column names are different, change them in my code.

In [1]:
#run this first!

#import neccessary libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = 'example app')

#the warnings were bothering me so i ask the code 2 ignore them
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
#replace variables & then run this code

#replace this variable with your path to training programs CSV
trainings = pd.read_csv("https://raw.githubusercontent.com/sejal234/heatmaps_by/main/data/trainings_full.csv", encoding = "ISO-8859-1")

#replace this variable with your path to schools CSV
schools = pd.read_csv("https://raw.githubusercontent.com/sejal234/heatmaps_by/main/data/CTD_sf_report.csv")

#replace this variable with the path in your computer you want to save the map to
save = '/Users/sejalgupta/Documents/heatmap_docs/trainings_schools_map_updated.html'

#uncomment to look at the data
#trainings.head()
#schools.head()

#uncomment to find the number of total schools, before deleting entries
print("Number of Schools:", len(schools))
print("Number of Trainings:", len(trainings))

Number of Schools: 61
Number of Trainings: 225


In [3]:
#run the code below!

def clean_schools(schools):
    #remove schools w/o addresses
    schools = schools.dropna(subset = ['School Address'])
    
    #edit out <br>
    schools = schools.replace('<br>',' ', regex=True)
    
    #find long, lat coordinates
    schools["loc"] = schools["School Address"].apply(geolocator.geocode)
    schools["point"]= schools["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
    
    #remove schools it did not find coordinates for 
        #(usually if address is incomplete)
    schools = schools.dropna(subset = ['point'])
    
    #split the .point into separate columns 'lat' 'lon' and 'altitude'
    schools[['lat', 'lon', 'altitude']] = pd.DataFrame(schools['point'].to_list(), index=schools.index)
    
    #keep just the columns i want
    schools = schools[["Career Test Drive Name", "School Address", "lat", "lon"]]
    
    #uncomment if you don't need
    print("Number of Schools:", len(schools))
    
    return schools

def clean_trainings(trainings):
    #theres a lot of repeats, need to only keep unique values
    trainings = trainings.drop_duplicates()
    
    #some don't have locations
    trainings = trainings.dropna(subset = ['Location (Latitude)'])
    
    #uncomment if you don't need
    print("Number of Trainings:", len(trainings))
    
    return trainings 
    
def make_map(schools, trainings, save):
    schools = clean_schools(schools)
    trainings = clean_trainings(trainings)
    htx = folium.Map(location=[29.749907, -95.358421], tiles = 'cartodbpositron', zoom_start=11)
    
    #add schools
    for i,r in schools.iterrows():
        location = (r["lat"], r["lon"])
        folium.Marker(location=location,
                          popup = r['Career Test Drive Name'],
                          tooltip=r["Career Test Drive Name"],
                     icon=folium.Icon(color = 'blue', icon = 'book'))\
        .add_to(htx)

    #add training programs
    for i,r in trainings.iterrows():
        #if the column names are diff in your input report, edit them below
        location = (r["Location (Latitude)"], r["Location (Longitude)"])
        folium.Marker(location=location,
                      popup = r['Training: Training Name'],
                      tooltip=r["Training: Training Name"],
                     icon=folium.Icon(color = 'red', icon = 'briefcase'))\
        .add_to(htx)
    htx.save(save)
    return htx

make_map(schools, trainings, save)

Number of Schools: 51
Number of Trainings: 221
